# CNN classifier

### In this file we are going to test the CNN classifier on the dataset.

The Goal is to answer the question that is the CNN able to detect the onBody devices from offBody.

And if so how? We will visualize the output of each layer to analyze how the optimizer extract features and potentially provide explanations of what those features are!

In [1]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import threading, os, sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # try to use CPU only

# addin path to import IQ module
sys.path.append('../')
import src.IQ as IQ


In [2]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/admin")
BLE = myclient["BLE"]

def query(collection, filter:dict, addFrameColumn=True):
    df =  pd.DataFrame(list(collection.find(filter)))
    if addFrameColumn:
        df['frame'] = df.apply(lambda x: x['I'] + np.dot(x['Q'],1j), axis=1)
    return df.copy()

In [3]:
# Create a StandardScaler object
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
scaler2 = MinMaxScaler(feature_range=(0, 1))
# Assuming 'new_csv_features' is your data

def to_complex(amplitudes, angles):
    return np.array([r * np.exp(1j * theta) for r, theta in zip(amplitudes, angles)])

def normalized(row):
    row = np.array(row)
    return scaler2.fit_transform(scaler.fit_transform(row.reshape(-1, 1)))

def fft_normalized(row, threshold = 0.5):
    tempfft = np.fft.fft(row)

    amp = np.abs(tempfft)
    angle = np.angle(tempfft)
    #filtering criteria
    filtering = amp > np.average(amp)*threshold
    
    amp[filtering] = 0
    angle[filtering] = 0
    temp = to_complex(amplitudes = amp,angles =  angle)
    ###### option 1 to go back to the time domain
    # temp = np.real(np.fft.ifft(temp))
    # # # uncomment to see the effect of frequency filtering
    # # plt.plot(row) 
    # # plt.plot(temp)
    # # plt.show()
    # # plt.close()
    # return normalized(temp)

    ###### option 2 to concat real and imaginary
    ##### try to teach the ifft at the same time
    return np.concatenate([normalized(np.real(temp)), normalized(np.imag(temp))])


In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)


filtering = {''}
df = query(BLE['onBody'], {'pos':'static','antenna_side':'left'})
min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:2000])
print(len(df['frame'][0]))


iq = IQ.IQ(Fc=2439810000+.1e4)

def configCreator(downSampleRate = 1, cutoff = 1e6):
    downSampleRate= max(downSampleRate, 1)
    return {                                      
            iq.gradient:{},
            iq.unwrapPhase:{},
            iq.phase:{}, 
            iq.butter:{'Fs': iq.Fs/downSampleRate, "cutoff": cutoff},
            iq.downSample:{'downSampleRate':downSampleRate, "shift": 0},
            iq.demodulate:{'Fs': iq.Fs},
           } 

methods = configCreator(downSampleRate=  1)
df['data'] = iq.apply(methods = methods, frame = df)

############### Defing and normalizing the input #############
# df['normalized_input_feature'] = df['data'].apply(lambda x: fft_normalized(x, threshold = 1))
df['normalized_input_feature'] = df['data'].apply(lambda x: normalized(x))
##################################################################################

X_train, X_test, y_train, y_test = train_test_split(df['normalized_input_feature'], df['dvc'], test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train.tolist())
X_test =  tf.convert_to_tensor(X_test.tolist())
y_train =  tf.convert_to_tensor(y_train.tolist())
y_test = tf.convert_to_tensor(y_test.tolist())

data_shape = len(df['data'][0])



2023-12-03 18:04:34.340800: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 18:04:34.365103: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 18:04:34.365129: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 18:04:34.365144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 18:04:34.369224: I tensorflow/core/platform/cpu_feature_g

2000


2023-12-03 18:04:44.883017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 18:04:44.883167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 18:04:44.883251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [5]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
# # Build the CNN model
# # input_data = Input(shape=(input_shape,))
# # encoded = Dense(encoding_dim + (input_shape - encoding_dim )//2, activation='relu')(input_data)
# # encoded = Dense(encoding_dim + (input_shape - encoding_dim )//4, activation='tanh')(encoded)
# model = Sequential()
# # model.add(Dense(100, activation='relu', input_shape=(data_shape,)))
# # model.input(shape=(X_train.shape[1], 1))
# model.add(Conv1D(filters=128, kernel_size=16, activation='relu', input_shape=(data_shape,)))
# model.add(Conv1D(filters=128, kernel_size=16, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(100, activation='sigmoid'))
# model.add(Dense(y_test_encoded.shape[1], activation='softmax'))  # 13 classes

model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(y_test_encoded.shape[1], activation='softmax'))  # 13 classes


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=64, batch_size=64, validation_data=(X_test, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {accuracy}')

Epoch 1/64


2023-12-03 18:27:01.713912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-03 18:27:02.912258: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-03 18:27:03.099103: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-03 18:27:03.618277: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f01acd15000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-03 18:27:03.618292: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-03 18:27:03.623629: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-03 18:27:03.680470: I ./tensorflow/compiler/jit/device_comp

37/37 [==============================] - 6s 24ms/step - loss: 2.2771 - accuracy: 0.2253 - val_loss: 1.6169 - val_accuracy: 0.3384
Epoch 2/64
37/37 [==============================] - 0s 12ms/step - loss: 1.4920 - accuracy: 0.3573 - val_loss: 1.3626 - val_accuracy: 0.4303
Epoch 3/64
37/37 [==============================] - 0s 12ms/step - loss: 1.3666 - accuracy: 0.4123 - val_loss: 1.3549 - val_accuracy: 0.4167
Epoch 4/64
37/37 [==============================] - 0s 12ms/step - loss: 1.3275 - accuracy: 0.4242 - val_loss: 1.2959 - val_accuracy: 0.4660
Epoch 5/64
37/37 [==============================] - 0s 12ms/step - loss: 1.2959 - accuracy: 0.4587 - val_loss: 1.2749 - val_accuracy: 0.4694
Epoch 6/64
37/37 [==============================] - 0s 12ms/step - loss: 1.2510 - accuracy: 0.4847 - val_loss: 1.2210 - val_accuracy: 0.5153
Epoch 7/64
37/37 [==============================] - 0s 12ms/step - loss: 1.1605 - accuracy: 0.5524 - val_loss: 1.1243 - val_accuracy: 0.5901
Epoch 8/64
37/37 [======

In [ ]:
z = model(X_test[0:100])
zz = pd.DataFrame(np.array(z))

# # num_rows = zz.shape[0]

# # # Plot each row
# # for i in range(num_rows):
# #     # plt.figure(figsize=(8, 4))  # Adjust the size as needed
# #   plt.plot(zz.iloc[i, :])

# # plt.show()
zz


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,2.210940e-06,0.000008,0.000046,0.006830,0.001110,0.000454,0.011084,0.000033,0.002209,0.000068,0.974120,0.003541,0.000494
1,1.779121e-06,0.000008,0.002859,0.044315,0.004439,0.933743,0.002776,0.000127,0.000845,0.002743,0.005991,0.000148,0.002006
2,4.004087e-06,0.000136,0.004579,0.040941,0.006156,0.018651,0.008958,0.000185,0.013918,0.002220,0.902648,0.000298,0.001306
3,7.926288e-07,0.000006,0.001689,0.029698,0.002899,0.955108,0.001790,0.000036,0.000963,0.000854,0.004685,0.000117,0.002154
4,2.524733e-07,0.000003,0.000341,0.034467,0.001138,0.960784,0.000899,0.000011,0.000206,0.000130,0.001381,0.000022,0.000617
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.521059e-06,0.001660,0.911694,0.000155,0.007675,0.000122,0.002399,0.000509,0.020495,0.038286,0.000929,0.004634,0.011441
96,1.521713e-05,0.000141,0.026068,0.107948,0.004056,0.052348,0.022734,0.000298,0.001987,0.037725,0.690642,0.037723,0.018314
97,1.369956e-05,0.000130,0.043163,0.044431,0.019566,0.698610,0.076379,0.000223,0.008797,0.011803,0.073950,0.002187,0.020748
98,2.863875e-06,0.000006,0.000110,0.000292,0.001443,0.000069,0.966615,0.000010,0.006892,0.000166,0.017879,0.005165,0.001351


In [ ]:
df = query(BLE['offBody'], {'pos':'1','antenna':'1', 'dvc':'7'})
# min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:min_length])
df['data'] = iq.apply(methods = methods, frame = df)

X_test = tf.convert_to_tensor(df['data'].tolist())

z = model(X_test)
zz = pd.DataFrame(np.array(z))

num_rows = zz.shape[0]

# Plot each row
for i in range(num_rows):
    # plt.figure(figsize=(8, 4))  # Adjust the size as needed
  plt.plot(zz.iloc[i, :])

plt.show()

ValueError: Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 2000, 1), found shape=(467, 4022)